In [78]:
import requests as r
import json
import pyttsx3
import os
from moviepy.editor import *

In [79]:
YOUTUBE_API_KEY = ""
VOICE_ENGINE = pyttsx3.init()
MIN_UPVOTES = 100
MIN_CHARACTERS = 1000
NUM_SHORTS_FOR_LONG_FORM = ...

In [80]:

def get_posts():
    with open("subreddits.txt", "r") as file:
        for sub in file.readlines():
            posts = r.get(f"https://www.reddit.com/r/{sub}/hot/.json", headers={"User-agent": "tester 0.1"}).content
            posts = json.loads(posts)["data"]["children"]
            posts = [p for p in posts
                     if len(p["data"]["selftext"]) >= MIN_CHARACTERS
                     and p["data"]["score"] >= MIN_UPVOTES
                     and p["data"]["media"] is None
                     and p["data"]["is_video"] is False]
    return posts


def get_tts(posts):
    for p in posts:
        title = p["data"]["title"].replace("/", "slash").replace("\\", "slash").replace(":", "").replace(".", "")
        text = f"r slash {p['data']['subreddit']}. {p['data']['selftext']}"
        text = text.replace("/", "slash").replace("\\", "slash")
        sentences = text.split(".")
        
        if not os.path.exists(f"./tts_files/{title}"):
            os.mkdir(f"./tts_files/{title}")
                
        for si, s in enumerate(sentences):
            if s == "": continue
            VOICE_ENGINE.save_to_file(s, f"./tts_files/{title}/sentence{si}.wav")
                             
        VOICE_ENGINE.runAndWait()
        
        
def generate_videos(posts):
    for p in posts:
        title = p["data"]["title"].replace("/", "slash").replace("\\", "slash").replace(":", "").replace(".", "")
        text = f"r slash {p['data']['subreddit']}. {p['data']['selftext']}"
        text = text.replace("/", "slash").replace("\\", "slash")
        sentences = text.split(".")

        background_clip = VideoFileClip(f"./background_videos/mc_parkour.mp4")
        dimension = background_clip.size
        
        subclips = []
        time_index = 0
        for si, s in enumerate(sentences):     
            if s == "": continue       
            audio_clip = AudioFileClip(f"./tts_files/{title}/sentence{si}.wav")
            duration = audio_clip.duration
            
            if time_index+duration >= background_clip.duration:
                time_index -= background_clip.duration
            
            background_subclip = background_clip.subclip(time_index, time_index+duration)
            text_clip = TextClip(s+".", size=dimension, font="Berlin-Sans-FB-Bold", color="white", fontsize=100, method="caption", align="center", interline=-50, stroke_color="black", stroke_width=10)
            
            subclip = CompositeVideoClip([background_subclip, text_clip])
            subclip = subclip.set_audio(audio_clip).set_duration(duration)
            subclips.append(subclip)
            
            time_index += duration
                        
        video = concatenate_videoclips(subclips)
        video.write_videofile(f"./videos/{title}.mp4")
    
        break 
                        
    
posts = get_posts()
get_tts(posts)
generate_videos(posts)

Moviepy - Building video ./videos/We have persistent scammers preying on this community.mp4.
MoviePy - Writing audio in We have persistent scammers preying on this communityTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./videos/We have persistent scammers preying on this community.mp4



t:  24%|██▍       | 1071/4388 [01:11<03:11, 17.34it/s, now=None]